## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [7]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [8]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [6]:
mlflow.get_experiment()

TypeError: get_experiment() missing 1 required positional argument: 'experiment_id'

In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2023/05/25 16:06:57 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: 'file:///C:/Users/joaov/Documents/_workspace/mlops-zoomcamp-jlima/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/860f2886a69e4f078bf8c3ed2c1c6658/artifacts'


In [ ]:
mlflow.list_experiments()

### Interacting with the model registry

In [10]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [11]:
client.list_registered_models()

AttributeError: 'MlflowClient' object has no attribute 'list_registered_models'

In [52]:
runs = mlflow.search_runs(experiment_ids='1')
run_id = runs.loc[0,'run_id']
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2023/05/25 16:22:43 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 2
Created version '2' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1685042563552, current_stage='None', description='', last_updated_timestamp=1685042563552, name='iris-classifier', run_id='860f2886a69e4f078bf8c3ed2c1c6658', run_link='', source='file:///C:/Users/joaov/Documents/_workspace/mlops-zoomcamp-jlima/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/860f2886a69e4f078bf8c3ed2c1c6658/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='2'>

['1', '0']